# ETL Processes
***
## Lists of tasks:
- ### <del>Consolidating Datasets</del>
- ### Normalising/Restructuring Tables
- ### Exploratory Data Analysis
- ### Data Cleaning
- ### Package ETL.py into a Class
***

## Content:
- ### [Consumer Dataset](#Consumer-dataset)
- ### [Transaction Dataset](#Transaction-dataset)
- ### [Merchant Dataset](#Merchant-dataset)


In [1]:
import pandas as pd
import numpy as np
import os
import re

# Set working directory
if not "/data/tables" in os.getcwd():
    os.chdir("../data/tables")

from pyspark.sql import SparkSession
from pyspark.shell import spark
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
import pyspark.sql.functions as F
import matplotlib.pyplot as plt

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.7.4 (default, Aug 13 2019 15:17:50)
Spark context Web UI available at http://olivers-macbook-pro.local:4040
Spark context available as 'sc' (master = local[*], app id = local-1661966109669).
SparkSession available as 'spark'.


# Consumer dataset

In [2]:
# Read csv file
consumer = spark.read.option("delimiter", "|").csv('tbl_consumer.csv', header = True)
consumer

name,address,state,postcode,gender,consumer_id
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
Lindsay Jimenez,00653 Davenport C...,NSW,2780,Female,154128
Rebecca Blanchard,9271 Michael Mano...,WA,6355,Female,712975
Karen Chapman,2706 Stewart Oval...,NSW,2033,Female,407340
Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
Stephen Williams,6804 Wright Crest...,WA,6056,Male,448088
Stephanie Reyes,5813 Denise Land ...,NSW,2482,Female,650435
Jillian Gonzales,461 Ryan Common S...,VIC,3220,Female,1058499


In [16]:
print(f"Dataset details: \n\tNumber of rows: {consumer.count()}", \
      f"\n\tNumber of distinct Consumer ID: {consumer.select('consumer_id').distinct().count()}", \
      f"\n\tNumber of distinct Postcodes: {consumer.select('postcode').distinct().count()}")

Dataset details: 
	Number of rows: 499999 
	Number of distinct Consumer ID: 499999 
	Number of distinct Postcodes: 3167


Note: 
- The **address field is fake** and derived from USA street names. We have included it to mimic a more realistic dataset, but the streets themselves are non-existent and if there are any matches, it will be a pure coincidence. <font color='red'>**Not sure what sort of information we can extract here if they are all fake</font> 
- The **postcode field is accurate** and should be **used for aggregated analysis** for joining with other geospatial datasets for demographic information (i.e ABS datasets) <font color='red'>**Highly relevant for geospatial analysis</font> 
- There is roughly a **uniform distribution at the state level** (i.e number of consumers per state is the same for all states).

# User detail dataset

In [3]:
user_detail = spark.read.parquet("consumer_user_details.parquet")
user_detail

user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975
6,407340
7,511685
8,448088
9,650435
10,1058499


In [12]:
print(f"Dataset details: \n\tNumber of rows: {user_detail.count()}", \
      f"\n\tNumber of distinct User ID: {user_detail.select('user_id').distinct().count()}", \
      f"\n\tNumber of distinct Consumer ID: {user_detail.select('consumer_id').distinct().count()}")

Dataset details: 
	Number of rows: 499999 
	Number of distinct User ID: 499999 
	Number of distinct Consumer ID: 499999


Note:
- Due to a difference between the internal system and a poor design choice (for some reason), the transaction tables use a **surrogate key** for each new user_id. <font color='red'>**Transaction dataset uses `user_id` to map customer but customer data are mapped to their own unique `customer_id` so the user detail data serves to map those two together</font> 
- However, the Consumer table has a **unique ID (some are missing on purpose)** field which will require some form of mapping between consumer_id to user_id. <font color='red'>**Might require further investigation and decide on whether it is appropriate to remove</font> 
- An additional mapping table has been provided to join the two datasets together.

***

# Transaction dataset

In [13]:
transaction = spark.read.parquet("transactions_20210228_20210827_snapshot/")

In [14]:
transaction

user_id,merchant_abn,dollar_value,order_id,order_datetime
18478,62191208634,63.255848959735246,949a63c8-29f7-4ab...,2021-08-20
2,15549624934,130.3505283105634,6a84c3cf-612a-457...,2021-08-20
18479,64403598239,120.15860593212783,b10dcc33-e53f-425...,2021-08-20
3,60956456424,136.6785200286976,0f09c5a5-784e-447...,2021-08-20
18479,94493496784,72.96316578355305,f6c78c1a-4600-4c5...,2021-08-20
3,76819856970,448.529684285612,5ace6a24-cdf0-4aa...,2021-08-20
18479,67609108741,86.4040605836911,d0e180f0-cb06-42a...,2021-08-20
3,34096466752,301.5793450525113,6fb1ff48-24bb-4f9...,2021-08-20
18482,70501974849,68.75486276223054,8505fb33-b69a-412...,2021-08-20
4,49891706470,48.89796461900801,ed11e477-b09f-4ae...,2021-08-20


In [25]:
transaction.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [24]:
min_date, max_date = transaction.select(min("order_datetime"), max("order_datetime")).first()

print(f"Dataset details: \n\tNumber of rows: {transaction.count()}", \
      f"\n\tNumber of distinct order: {transaction.select('order_id').distinct().count()}", \
      f"\n\tPeriod: {min_date} - {max_date}")

Dataset details: 
	Number of rows: 3643266 
	Number of distinct order: 3643266 
	Period: 2021-02-28 - 2021-08-27


In [ ]:
path = "transactions_20210228_20210827_snapshot/"
list_files = os.listdir(path)
list_files = list_files[1:(len(list_files)-1)]
list_files

In [ ]:
# import modules
from pyspark.sql import SparkSession
import functools
 
# explicit function
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

# insert datetime
file_name = os.listdir(path+ list_files[0])[1]
transaction = spark.read.parquet(path+ list_files[0] +"/" + file_name)
for i in list_files[1:]:
    file_name = os.listdir(path + i)[1]
    tmp = spark.read.parquet(path+ i +"/" + file_name)
    transaction = unionAll([transaction, tmp] )

In [ ]:
transaction.count()

# Merchant dataset

In [26]:
merchant = spark.read.parquet("tbl_merchants.parquet")
merchant

name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162
Fusce Company,"[(gift, card, nov...",10206519221
Aliquam Enim Inco...,"[(computers, comP...",10255988167
Ipsum Primis Ltd,"[[watch, clock, a...",10264435225
Pede Ultrices Ind...,([computer progra...,10279061213
Nunc Inc.,"[(furniture, home...",10323485998


In [27]:
merchant.printSchema()

root
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: long (nullable = true)



In [34]:
print(f"Dataset details: \n\tNumber of rows: {merchant.count()}", \
      f"\n\tNumber of distinct Merchant ABN: {merchant.select('merchant_abn').distinct().count()}")

Dataset details: 
	Number of rows: 4026 
	Number of distinct Merchant ABN: 4026


In [28]:
# Display top 5 rows for "tags"
merchant.select("tags").head(5)


[Row(tags='((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))'),
 Row(tags='([cable, satellite, and otHer pay television and radio services], [b], [take rate: 4.22])'),
 Row(tags='([jewelry, watch, clock, and silverware shops], [b], [take rate: 4.40])'),
 Row(tags='([wAtch, clock, and jewelry repair shops], [b], [take rate: 3.29])'),
 Row(tags='([music shops - musical instruments, pianos, and sheet music], [a], [take rate: 6.33])')]

### The tags column consists of tags, revenue levels and take rate of a merchant
- **Revenue Levels**: (a, b, c, d, e) represents the **level of revenue bands** (unknown to groups). a denotes the smallest band whilst e denotes the highest revenue band. <font color='red'>**Highly relevant in ranking merchant</font> 
- **Take Rate**: the **fee charged by the BNPL firm** to a merchant on a transaction. That is, for each transaction made, a certain percentage is taken by the BNPL firm.<font color='red'>**Highly relevant in ranking merchant</font> 
- The dataset has been created to mimic a Salesforce data extract (i.e salespeople will type in tags and segments within a **free-text** field). <font color='red'>This suggests use of lemmatizating/stemming/fuzzy methods to group similar texts?</font> 
- As such, please be aware of small **human errors** when parsing the dataset.
- For Example, the tag field may have errors as they were manually input by employees.

### Extract Revenue Levels and Take Rate columns

In [30]:
# Function to extract tags, revenue level and take rate from tags column
def extract_tags(arr, category='tags'):
    
    # Split tags into the three components
    arr = arr[1:-1]
    split_arr = re.split('\), \(|\], \[', arr.strip('[()]'))
    
    if category == 'take_rate':
        return re.findall('[\d\.\d]+', split_arr[2])[0]
    
    elif category == 'revenue_level':
        return split_arr[1].lower()
    
    return split_arr[0].lower()

extract_tag_udf = udf(lambda x: extract_tags(x, 'tags'))
extract_tr_udf = udf(lambda x: extract_tags(x, 'take_rate'))
extract_rev_udf = udf(lambda x: extract_tags(x, 'revenue_level'))


In [ ]:
# Extract all three components in tags as standalone columns
merchant = merchant.withColumn("tags_only",
                               extract_tag_udf(F.col('tags')))

merchant = merchant.withColumn("take_rate",
                               extract_tr_udf(F.col('tags')))

merchant = merchant.withColumn("revenue_level",
                               extract_rev_udf(F.col('tags')))
